In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    model="gpt-5-nano",
    temperature=1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "movie": "The Godfather",
        "answer": """
Here is what I know:
Title: The Godfather
Director: Francis Ford Coppola
Stars: Marlon Brando, Al Pacino, James Caan
Production Budget: around $7 million
Worldwide Gross: over $270 million
Genres: Crime, Drama
Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
""",
    },
    {
        "movie": "Pulp Fiction",
        "answer": """
Here is what I know:
Title: Pulp Fiction
Director: Quentin Tarantino
Stars: Tim Roth, Amanda Plummer, Laura Lovelace
Production Budget: around $8 million
Worldwide Gross: over $212 million
Genres: Crime, Drama
Synopsis: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.
""",
    },
    {
        "movie": "Forrest Gump",
        "answer": """
Here is what I know:
Title: Forrest Gump
Director: Robert Zemeckis
Stars: Tom Hanks, Rebecca Williams, Sally Field
Production Budget: around $55 million
Worldwide Gross: over $679 million
Genres: Drama, Romance
Synopsis: The history of the United States from the 1950s to the '70s unfolds from the perspective of an Alabama man with an IQ of 75, who yearns to be reunited with his childhood sweetheart.
""",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie imformation expert, you give short answers."),
        few_shot_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Inception"})


Here is what I know:
Title: Inception
Director: Christopher Nolan
Stars: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page
Production Budget: around $160 million
Worldwide Gross: over $829 million
Genres: Action, Adventure, Sci-Fi
Synopsis: A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.

AIMessageChunk(content='Here is what I know:\nTitle: Inception\nDirector: Christopher Nolan\nStars: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page\nProduction Budget: around $160 million\nWorldwide Gross: over $829 million\nGenres: Action, Adventure, Sci-Fi\nSynopsis: A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.')